## Script for TIF file
#### Convert TIF to PNG
#### Save tile PNG image

In [18]:
import cv2
import numpy as np
import os

from scipy import signal

def convert_img(img, pmin=80, pmax=98):
    """Convert image to 8bit file
    
    Convert image to 8bit file and visualize it for labeling
    
    Args:
        img(ndarray): image file array
        pmin(int): percentile of minimum value. Default: 80 %
        pmax(int): percentile of maximum value. Default: 98 %
    
    Returns:
        img_8bit_med(ndarray): visualized image file array
    """
    # cut values outside of pmin% ~ pmax% of image value
    percentile = np.nanpercentile(img, [pmin, pmax])
    img[img < percentile[0]] = percentile[0]
    img[img > percentile[1]] = percentile[1]    

    img_max = np.max(img)
    img_min = np.min(img)
    img_8bit = np.uint8((img - img_min) / (img_max - img_min) * 255)
    img_8bit_med = signal.medfilt2d(img_8bit, kernel_size=3)

    return img_8bit_med

def save_tile_img(img, save_home, base_name='', size=1024, ovr=10, is_apply_pad=True):
    """Save image into tile image
    
    Args:
        img(ndarray): image file array
        save_home(str): Save path to directory
        base_name(str): Base name of tile images. E.g., {base_name}_xmin_ymin_xmax_ymax.png
        size(int): size of tile image. Image is square.
        ovr(int): overap ratio of adjacent tile image. Default: 10 %
        is_apply_pad (bool): Whether to apply padding area. Default: True
    """
    print('Saving tile image...')
    
    assert os.path.isdir(save_home), '{} not exists'.format(save_home)        
    
    az, rg = img.shape
    assert az >= size, 'Given image {} is smaller than tile image {}'.format(img.shape, size)
    assert rg >= size, 'Given image {} is smaller than tile image {}'.format(img.shape, size)    
    
    for a in range(0, az, size):
        print('Processing {} / {} line...'.format(a, az))
        ymin = a
        ymax = ymin + size
             
        if ymax >= az:
            ymax = az            
            if not is_apply_pad:
                ymin = az - patch_size
                    
        for r in range(0, rg, size):
            xmin = r
            xmax = xmin + size
            
            if xmax > rg:                
                xmax = rg
                if not is_apply_pad:
                    xmin = rg - patch_size
                
            if is_apply_pad:
                patch_img = np.zeros([size, size])
                patch_img[:ymax - ymin, :xmax - xmin] = img[ymin:ymax, xmin:xmax]
            else:
                patch_img = img[ymin:ymax, xmin:xmax]
                
            img_name = base_name + '_{}_{}_{}_{}.png'.format(xmin, ymin, xmax, ymax)
            assert cv2.imwrite(os.path.join(save_home, img_name), patch_img)
            
    print('Done')

In [7]:
import PIL
import numpy as np

from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000  # None is also possible

img = Image.open('img_tif/s1a-iw-grd-vv-20210712t095500-20210712t095525-038743-04925c-001.tiff')
img = np.array(img, dtype=np.float64)

In [19]:
intensity_cvt = convert_img(img, pmin=80, pmax=98)
save_home = 'img_tif/patch'
tiff_name = '20210712t095500_VV'
save_tile_img(intensity_cvt, save_home, tiff_name, size=1024, ovr=10)

Saving tile image...
Processing 0 / 16718 line...
Processing 1024 / 16718 line...
Processing 2048 / 16718 line...
Processing 3072 / 16718 line...
Processing 4096 / 16718 line...
Processing 5120 / 16718 line...
Processing 6144 / 16718 line...
Processing 7168 / 16718 line...
Processing 8192 / 16718 line...
Processing 9216 / 16718 line...
Processing 10240 / 16718 line...
Processing 11264 / 16718 line...
Processing 12288 / 16718 line...
Processing 13312 / 16718 line...
Processing 14336 / 16718 line...
Processing 15360 / 16718 line...
Processing 16384 / 16718 line...
Done


In [20]:
img.shape

(16718, 25502)